# Bias wave stability

Looking at the impact of sequencer changes on the bias wave\
and large variations at the beginning of the serial read.\
Craig Lage - 07-Feb-23 - Sequencer files were created by Yousuke Utsumi and Stuart Marshall

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.ip.isr import IsrTask, IsrTaskConfig
import lsst.afw.display as afwDisplay
import lsst.geom as geom

In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])

## First, we look at the impact of the more conventional sequencer changes

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.suptitle("AuxTel bias stability")
names = ['Old Baseline', '20230509', '20230510', '20230511', '20230515']
xplot = np.arange(509)
for i, expId in enumerate([2023022200100, 2023050900020, 2023051000565, 2023051100565, 2023051500015]):
    exp = butler.get('raw', detector=0, exposure=expId)
    isrResult = isrTask.run(exp)
    plotCounter = 1
    for amp in exp.getDetector().getAmplifiers():
        ampName = amp.getName()
        bbox = amp.getBBox()
        subExp = isrResult.exposure[bbox]
        arr = subExp.image.array
        plt.subplot(4,4,plotCounter)
        plt.title(ampName)
        plt.plot(xplot, np.mean(arr[100:1900, :], axis=0), label=names[i])
        plt.ylim(-20,20)
        if plotCounter % 4 == 1:
            plt.ylabel("Signal(ADU)")
        else:
            plt.yticks([])
        if plotCounter > 12:
            plt.xlabel("Xcoordinate(pixels)")
        else:
            plt.xticks([])
        plotCounter += 1
plt.subplot(4,4,1)
plt.legend(loc='lower left', fontsize=5)
plt.savefig("/home/c/cslage/u/AuxTel/isr/sequencer_tests_22feb23/Bias_Stability_Sequencer_15May23.pdf")